# 🎶 Objectif 4 : Visualisation dynamique des Streams Musicaux : Évolution Temporelle, influence des saisons de l'année, impact des collaborations

#### Rappel

L'objectif est de visualiser l'évolution du nombre de streams musicaux sur un siècle avec une triple granularité de temps permettant d'explorer les tendances par saison (hiver, printemps, été, automne). Le graphique à ligne permet de se concentrer sur des périodes spécifiques, offrant ainsi une vue détaillée de l'activité musicale au fil du temps. Un bonus de cette analyse permet d'observer l'évolution du nombre d'artistes impliqués dans chaque morceau pendant ces périodes. Ce graphique est conçu pour mettre en évidence les moments les plus dynamiques du secteur musical.

**Techniques utilisées :**  
- **Triple granularité de temps** : Utilisation des colonnes année, mois, jour pour offrir une vue détaillée.  
- **Visualisation de la tendance des streams** : Diagramme en ligne avec possibilité de zoomer sur des périodes spécifiques (focus + context).  
- **Affichage de la collaboration artistique** : Intégration du nombre d'artistes associés à chaque morceau pour via la multiplication d'un coefficient sur le diamètre de la bulle affichée.


## Import et configuration

In [9]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np

## Chargement et préparation des données


In [10]:
# Load and prepare data
df = pd.read_csv('dataset/dataset_filtered.csv')
df['streams'] = pd.to_numeric(df['streams'], errors='coerce')
df['date'] = pd.to_datetime(dict(
    year=df['released_year'],
    month=df['released_month'],
    day=df['released_day']
))

# Filter for tracks with >10M streams
popular_tracks = df.groupby('track')['streams'].sum() >= 10_000_000
df_filtered = df[df['track'].isin(popular_tracks[popular_tracks].index)]

# Aggregate monthly streams
monthly_streams = df_filtered.groupby(
    ['date', 'track', 'artist_name', 'artist_count']
)['streams'].sum().reset_index()



# Analyse Saisonnière des Streams Musicaux 🎶

Pour mieux comprendre les périodes où le secteur musical est le plus dynamique, nous avons catégorisé les données par saison. 
Chaque saison est représentée par une couleur, un emoji, et les codes couleur correspondants :

- **❄️ Hiver** : ![#870000](https://via.placeholder.com/15/87ceeb/000000?text=+)   
- **🌱 Printemps** : ![#8FCE00](https://via.placeholder.com/15/8FCE00/000000?text=+)  
- **🏖️ Été** : ![#FFCE3A](https://via.placeholder.com/15/FFCE3A/000000?text=+) 
- **🍂 Automne** : ![#CE7E00](https://via.placeholder.com/15/CE7E00/000000?text=+)

Cette visualisation saisonnière permet de mettre en évidence les périodes clés où le nombre de streams atteint son apogée, offrant un aperçu des tendances musicales en fonction des saisons.





In [11]:
def get_season_color(date):
    month = date.month
    if month in [12, 1, 2]:
        return 'rgb(135,206,235)'  # Winter
    elif month in [3, 4, 5]:
        return 'rgb(143,206,0)'    # Spring
    elif month in [6, 7, 8]:
        return 'rgb(255,206,58)'   # Summer
    return 'rgb(206,126,0)'        # Autumn

### 🧮 Calcul des tailles des Marqueurs en Fonction du Nombre d'Artistes 🎨:


In [12]:
# Calculate visualization parameters
monthly_streams['color'] = monthly_streams['date'].apply(get_season_color)
artist_count_range = monthly_streams['artist_count'].agg(['min', 'max'])
marker_sizes = 10 + (monthly_streams['artist_count'] - artist_count_range['min']) / (
    artist_count_range['max'] - artist_count_range['min']
) * 30

In [13]:



# Create figure
fig = go.Figure()

# Add scatter plot
fig.add_trace(go.Scatter(
    x=monthly_streams['date'],
    y=monthly_streams['streams'],
    mode='markers',
    marker=dict(
        color=monthly_streams['color'],
        size=marker_sizes,
        opacity=0.7,
        line=dict(width=1, color='#2c3e50')
    ),
    hovertemplate=(
        "Date: %{x|%d/%m/%Y}<br>"
        "Streams: %{y:,.0f}<br>"
        "Titre: %{customdata[0]}<br>"
        "Artiste: %{customdata[1]}<br>"
        "Nombre d'artistes: %{customdata[2]}<extra></extra>"
    ),
    customdata=monthly_streams[['track', 'artist_name', 'artist_count']].values
))

# Add season legend
seasons = [
    ("Hiver", 'rgb(160,210,255)'),
    ("Printemps", 'rgb(143,206,0)'),
    ("Été", 'rgb(255,206,58)'),
    ("Automne", 'rgb(206,126,0)')
]

for i, (name, color) in enumerate(seasons):
    fig.add_shape(
        type="rect",
        x0=-0.24, x1=-0.2,
        y0=1.05 - i * 0.12,
        y1=1.0 - i * 0.12,
        fillcolor=color,
        line=dict(width=0),
        xref="paper", yref="paper"
    )
    fig.add_annotation(
        text=name,
        x=-0.18, y=1.025 - i * 0.12,
        xref="paper", yref="paper",
        showarrow=False,
        font=dict(color='white', size=12),
        align="left"
    )

# Update layout
fig.update_layout(
    title=dict(
        text="Dynamique des streams au long de l'histoire: Influence des saisons et des artistes",
        font=dict(size=20, color='white'),
        x=0.5, y=0.95
    ),
    xaxis=dict(
        title="Date",
        gridcolor='gray',
        showgrid=True,
        tickformat='%d/%m/%Y',
        range=['2020-01-01', monthly_streams['date'].max().strftime('%Y-%m-%d')],
        color='white'
    ),
    yaxis=dict(
        title="Nombre de streams",
        gridcolor='gray',
        showgrid=True,
        type='log',
        range=[np.log10(10_000_000), np.log10(3_800_000_000)],
        color='white'
    ),
    template='plotly_dark',
    plot_bgcolor='black',
    paper_bgcolor='black',
    hovermode='closest',
    height=800,
    width=1200,
    margin=dict(t=100, l=250, r=50, b=50)
)

fig.show()